In [1]:
#| default_exp train
#| default_cls_lvl 3

In [2]:
#| hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# train

> TODO

In [3]:
#| export
import argparse, os
from pathlib import Path
import warnings

import pandas as pd
from transformers import logging

from kaggle_comp import config, train_dispatcher, utils
from kaggle_comp.config import CFG

# silence all the HF warnings
warnings.simplefilter("ignore")
logging.set_verbosity_error()
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [4]:
#| hide_input
import pdb
from fastcore.test import *

## Train a fold

In [5]:
#| export
def run_fold(
    CFG,
    n_fold: int,
    experiment_name: str,
    run_id: str,
    grid_id: str,
    train_data=config.TRAINING_FILE,
    verbose: bool = True,
):

    train_config = train_dispatcher.experiments[experiment_name]
    model_name = f"{experiment_name}_{run_id}_{grid_id}_fold_{n_fold}"

    comp_trainer_cls = train_config["comp_trainer_cls"]
    comp_trainer = comp_trainer_cls(
        train_config=train_config, model_name=model_name, model_output_path=config.MODEL_OUTPUT, log_output_path=config.LOG_OUTPUT
    )

    if isinstance(train_data, pd.DataFrame):
        train_df = train_data.copy()
    else:
        train_df = pd.read_csv(train_data)
    
    train_df["is_valid"] = train_df["k_fold"] == n_fold

    log_df, oof_df = comp_trainer.train(
        CFG,
        train_df,
        n_fold=n_fold,
        run_id=run_id,
        grid_id=grid_id,
        experiment_name=experiment_name,
        verbose=verbose,
    )

    oof_preds = oof_df[[col for col in oof_df.columns if col.startswith("pred")]].values
    oof_targs = oof_df[[col for col in oof_df.columns if col.startswith("targ")]].values

    score = utils.comp_metric_score(oof_preds, oof_targs)

    if verbose:
        print("--- logging results ---")

    log_df.to_csv(Path(config.LOG_OUTPUT) / str(model_name + ".csv"), index=False)
    oof_df.to_csv(Path(config.LOG_OUTPUT) / str("oof_" + model_name + ".csv"), index=False)

    if verbose:
        print(f"Fold={n_fold}, Score={score}")


In [6]:
#| export
if __name__ == "__main__" and utils.run_env != "local_nb":
    # instantiate argparser
    parser = argparse.ArgumentParser()

    # define args
    parser.add_argument("--experiment_name", type=str)
    parser.add_argument("--n_fold", type=int, default=0)
    parser.add_argument("--run_id", type=str, default="dummy")
    parser.add_argument("--grid_id", type=int, default=0)
    parser.add_argument("--verbose", type=str, default=True)

    # read in args from terminal
    args = parser.parse_args()

    run_fold(
        CFG=CFG,
        n_fold=args.n_fold,
        experiment_name=args.experiment_name,
        run_id=args.run_id,
        grid_id=args.grid_id,
        train_data=config.TRAINING_FILE,
        verbose=args.verbose,
    )


In [7]:
class CFG:
    subset = 1.0
    n_fold = 5
    fold = [0, 1]
    random_seed = 42
    batch_size = 4
    use_fp16 = False
    n_unfrozen_epochs = 10
    #strat_feat = "section_scores"
    preprocess = "basic"
    postprocess = "none"
    augment = "none"
    train_folds = "train_folds.csv"
    experiments = ['blurr_deberta_v3_small']
    grid_params = ['experiments', 'fold']


In [8]:
# manual debugging check
run_fold(CFG=CFG, n_fold = 0, experiment_name = "blurr_deberta_v3_small", run_id = "2022_01_01_01_01", grid_id = 0)

Experiment: blurr_deberta_v3_small
Run: 2022_01_01_01_01 | Grid ID: 0 | Fold: 0
Training config: f{'comp_trainer_cls': <class 'kaggle_comp.framework_utils.fastai.FastaiCompTrainer'>, 'model_checkpoint': 'microsoft/deberta-v3-small', 'hf_config_kwargs': {'num_labels': 6, 'cls_dropout': 0.15, 'pooler_dropout': 0.15, 'hidden_dropout_prob': 0.05, 'attention_probs_dropout_prob': 0.1, 'layer_norm_eps': 1e-05}, 'hf_tokenizer_kwargs': {}, 'anchor_col': 'anchor', 'target_col': 'target', 'max_length': 140, 'include_labels': True, 'tok_kwargs': {}, 'batch_size': 128, 'adam_beta1': 0.9, 'adam_beta2': 0.995, 'adam_eps': 1e-06, 'weight_decay': 0.01, 'max_grad_norm': None, 'save_best_model': True, 'use_fp16': True, 'n_frozen_epochs': 0, 'frozen_lr': None, 'n_unfrozen_epochs': 10, 'unfrozen_lrs': [1e-05, 0.02]}
Building HF objects ...
Building DataLoaders ...
b0 shape: torch.Size([4, 140]); b1 shape: torch.Size([4, 6])
Building Learner ...


epoch,train_loss,valid_loss,mcrmse,time
0,0.607733,0.617320,0.649769,02:00
1,0.639494,0.587982,0.621590,02:01
2,0.554379,0.503882,0.535492,02:01
3,0.540050,0.531948,0.565210,02:02
4,0.498622,0.511245,0.541444,02:02
5,0.449274,0.507042,0.537143,02:00
6,0.409575,0.529161,0.563487,02:00
7,0.379209,0.490963,0.520980,02:01
8,0.357364,0.488502,0.518009,02:01
9,0.340436,0.488603,0.518625,02:01


Better model found at epoch 0 with mcrmse value: 0.6497691869735718.
Better model found at epoch 1 with mcrmse value: 0.6215897798538208.
Better model found at epoch 2 with mcrmse value: 0.5354924201965332.
Better model found at epoch 7 with mcrmse value: 0.5209799408912659.
Better model found at epoch 8 with mcrmse value: 0.5180088877677917.


--- saving model ---
--- logging results ---
Fold=0, Score=0.5180088877677917


## Export -

In [18]:
#| hide
import nbdev

nbdev.nbdev_export()